# Working with Callback Functions

MIP solvers are complicated combinations of many techniques: cutting planes, heuristics, branching rules, etc.

Some solvers allow you to customize aspects of the solve process in a deeper way than just setting options for these parameters. You can provide code to be run when certain events happen, and the solver **calls back** to these functions to ask what action(s) should be taken. Why might you want to do this? Some situations:

* you might want to extract the intermediate solutions found in the branch&bound tree, or other useful information during the solution process, and not work only with the optimal solution.

* you might not want to enumerate all of your constraints at the onset, and enforce certain constraints only when they're needed.

We'll explore these cases in this notebook.

## Warm-Up: Linear Regression and its variants

In [1]:
srand(123)

n = 20
p = 5

real_x = 10*rand(p)
A = rand(n,p)
b = A*real_x + rand(n);

We start with linear regression: suppose we have a data matrix $A$, and vector $b$, and we want to estimate a vector $x$ such that:

$$
\underset{x}{\min}\ || Ax-b ||_2^2
$$

One way of expressing it in JuMP is through the following formulation:

$$
\underset{x}{\min}\ \sum_i z_i^2 \\
\text{s.t.}\quad z_i = a_i^\top x - b_i \quad\forall i
$$

or equivalently:

$$
\underset{x}{\min}\ \sum_i z_i^2 \\
\text{s.t.}\quad z = Ax - b
$$

where $a_i^\top$ is the $i$-th row of the data matrix $A$.

In [2]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, z[1:n])

@objective(m, Min, sum(z[i]^2 for i in 1:n))

@constraint(m, z .== A*x-b)
        
solve(m)

Optimize a model with 20 rows, 25 columns and 120 nonzeros
Model has 20 quadratic objective terms
Coefficient statistics:
  Matrix range     [9e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 2e+01]
Presolve time: 0.00s
Presolved: 20 rows, 25 columns, 120 nonzeros
Presolved model has 20 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.900e+02
 Factor NZ  : 2.100e+02
 Factor Ops : 2.870e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.67645245e+06 -4.99435609e+06  3.47e+03 0.00e+00  9.84e+05     0s
   1   1.40930140e+06 -1.58740897e+06  3.47e-03 0.00e+00  1.20e+05     0s
   2   1.86695636e+05 -2.51501497e+05  3.47e-09 0.00e+00  1.75e+04     0s
   3   2.08575874e+04 -4.24957571e+04  3.91e-14 2.84e-14  2.53e+03     0s
   4 

:Optimal

In [3]:
getvalue(x)

5-element Array{Float64,1}:
 7.85979
 9.8411 
 6.95982
 3.84803
 3.20848

In [4]:
A \ b

5-element Array{Float64,1}:
 7.85979
 9.8411 
 6.95982
 3.84803
 3.20848

In [5]:
real_x

5-element Array{Float64,1}:
 7.68448
 9.40515
 6.73959
 3.95453
 3.13244

**Discussion**: When might you want to do this yourself (in JuMP/etc), versus using a opensource/commercial package?

>**\[Exercise\]**: Non-negative Least Squares

> How might we modify the formulation above to solve for non-negative least squares, i.e. $\underset{x\geq 0}{\min}\ || Ax-b ||_2^2$

>**\[Exercise\]**: Sparse Linear Regression

> How might we modify the formulation above if we know at most $k$ of the coefficients are non-zero? i.e. $\underset{x: ||x||_0\leq k}{\min}\ || Ax-b ||_2^2$

**Discussion**: Do you know of other ways of getting sparse (as in low number of non-zero coefficients) solutions in a linear regression setting?

## Extracting Intermediate Solutions
Rather than having to parse the solver log, sometimes querying for information (amongst other things) might be useful for you to do convergence plots (or perform other diagnostics). Informational callbacks are added to a JuMP model with the `addinfocallback(m::Model, f::Function; when::Symbol)` function, where the `when` argument should be one of `:MIPNode`, `:MIPSol` or `:Intermediate` (listed under `cbgetstate()` in the [MathProgBase documentation](https://mathprogbasejl.readthedocs.io/en/latest/callbacks.html)).

Suppose we wish to extract all the incumbent solutions generated during the branch&bound process. Here's how we can modify the code above to do so:

In [6]:
using JuMP, Gurobi

M = 10000
k = 2

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, y[1:p], Bin)
@variable(m, z[1:n])

@objective(m, Min, sum(z[i]^2 for i in 1:n))

@constraint(m, z .== A*x - b)
@constraint(m,[j=1:p], x[j] <= M*y[j])
@constraint(m,[j=1:p], x[j] >= -M*y[j])
@constraint(m, sum(y[j] for j in 1:p) <= k)

### --- NEW ---
solutionvalues = []
function solncallback(cb)
    push!(solutionvalues, JuMP.getvalue(x))
end
addinfocallback(m, solncallback, when = :MIPSol)
### END OF NEW

solve(m)

Optimize a model with 31 rows, 30 columns and 145 nonzeros
Model has 20 quadratic objective terms
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [9e-03, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 26 rows, 30 columns, 135 nonzeros
Presolved model has 20 quadratic objective terms
Variable types: 25 continuous, 5 integer (5 binary)

Root relaxation: objective 2.196497e+00, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.19650    0    5          -    2.19650      -     -    0s
H    0     0                     415.3988979    2.19650  99.5%     -    0s
H    0     0                     356.2185398    2.19650  99.4%     -    0s
    

We can inspect the incumbent solutions generated:

In [7]:
solutionvalues

3-element Array{Any,1}:
 [0.0,0.0,0.0,13.563,11.1933] 
 [13.5941,16.1585,0.0,0.0,0.0]
 [0.0,17.6869,12.7185,0.0,0.0]

As well as their corresponding objective values:

In [8]:
[norm(A*sol_x-b)^2 for sol_x in solutionvalues]

3-element Array{Float64,1}:
 415.399
 356.219
 189.036

**Remark**: Can you identify the objective values of the incumbent solution in the solver log above? Which of the incumbent solutions were found via heuristics, and which ones were found via branch&bound?

>**\[Exercise\]**: Information Callbacks

> How might you organize the code such that you can extract

> - the objective function value of the current solution
> - the current best bound on the optimal solution
> - the current time taken since the start of the solution process

> (Hint: http://www.juliaopt.org/JuMP.jl/0.15/callbacks.html#informational-callbacks)

## Modelling using Lazy Constraints

### Motivation
**Question**: Now suppose we wish to solve the sparse linear regression problem

$$
\underset{x}{\min}\ \sum_i z_i^2 \\
\text{s.t.}\quad z_i = a_i^\top x - b \quad\forall i \\
x_j \leq My_j \\
x_j \geq -My_j \\
\sum_{j=1}^p y_j \leq k \\
y_j\in\{0,1\}\quad\forall j=1,\dots,p
$$

but as a sequence of mixed integer linear programs, rather than a single mixed integer quadratic program. How might we go about doing so?

**Answer**: ![](outerapprox.jpg)
(image source: http://www.mit.edu/~dimitrib/Polyhedral_Approx_Tsinghua.pdf)

In general, we might want to perform a "linearization" of some (often convex) function, where we perform a succession of "outer-approximations" that converges towards the underlying function we wish to optimize over.

**Remark**: For readers who are interested in cutting-plane methods, the following might be a good reference: https://web.stanford.edu/class/ee392o/localization-methods.pdf

**Discussion**: What are potential issues we might run into, if we wish to implement this?

### Description
Lazy constraints are useful when the full set of constraints is too large to explicitly include in the initial formulation. We might want to modify the MIP solution process such that when we have a new solution (for example with a heuristic or by solving a problem at a node in the branch-and-bound tree), we are given the opportunity to generate additional constraint(s). This allows us to generate them only upon demand, and stop the process based on our termination criteria.

There are three important steps to providing a lazy constraint callback:

1. we must write a function that will analyze the current solution that takes a single argument, e.g. `mylazycongenerator(cb)`, where cb is a reference to the callback management code inside JuMP.
2. do whatever analysis of the solution you need to inside your function to generate the new constraint before adding it to the model with `@lazyconstraint(cb, myconstraint)` (instead of the usual `@constraint(m, myconstraint)`).
3. finally we notify JuMP that this function should be used for lazy constraint generation using the `addlazycallback(m, mylazycongenerator)` function before we call `solve(m)`.

For more information, see http://www.juliaopt.org/JuMP.jl/0.15/callbacks.html#lazy-constraints

### Task
Now, we'd like to implementing sparse linear regression using lazy constraints, instead of formulating it as a quadratic optimization problem. To do so, we first represent the objective function

$$
f(z) = ||z||_2^2
$$

as the pointwise maximum of affine functions:

$$
f(z) = \underset{\beta}{\sup} ||\beta||_2^2 + 2\beta^\top(z-\beta)
$$

(or equivalently)

$$
f(z) = \min\ \eta \\
\text{s.t.}\ \ \eta \geq ||\beta||_2^2 + 2\beta^\top(z-\beta) \quad\forall\beta
$$

## Code
So our JuMP model becomes:

$$
\underset{x}{\min}\ \eta \\
\text{s.t.}\quad 
\eta \geq ||\beta||_2^2 + 2\beta^\top(z-\beta) \quad\forall \beta \\
z_i = a_i^\top x - b \quad\forall i \\
x_j \leq My_j \\
x_j \geq -My_j \\
\sum_{j=1}^p y_j \leq k \\
y_j\in\{0,1\}\quad\forall j=1,\dots,p
$$

where we represent the set of (infinite) constraints:

$$
\eta \geq ||\beta||_2^2 + 2\beta^\top(z-\beta) \quad\forall \beta
$$

using lazy constraints.

>**\[Exercise\]**: Lazy Least Squares

> Implement the above formulation in JuMP (reference: http://www.juliaopt.org/JuMP.jl/0.15/callbacks.html#lazy-constraints)

**Discussion**: So far, we've been looking at modelling a convex function as the pointwise maximum of a (possibly infinite) set of affine functions in the objective function. Can you anticipate other optimization problems that might be usefully modelled using a large (possibly infinite) number of linear constraints?

See for e.g. Robust Portfolio Optimization and Travelling Salesman by Iain in [last year's IAP class](https://github.com/joehuchette/OR-software-tools-2015/blob/master/7-adv-optimization/Callbacks.ipynb).

**Discussion**: Sometimes a good polyhedral outer approximation might need too many linear constraints, and might benefit from "extended formulations". See http://www.mit.edu/~mlubin/micp-cribb.pdf (from slides 19 onwards) for some details.

>**\[Solution\]**: Non-negative Least Squares

In [11]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, z[1:n])

@objective(m, Min, sum(z[i]^2 for i in 1:n))

@constraint(m, [i=1:n], z[i] == sum(A[i,j]*x[j] for j in 1:p) - b[i])
        
solve(m)

Optimize a model with 20 rows, 25 columns and 120 nonzeros
Model has 20 quadratic objective terms
Coefficient statistics:
  Matrix range     [9e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 2e+01]
Presolve time: 0.00s
Presolved: 20 rows, 25 columns, 120 nonzeros
Presolved model has 20 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.900e+02
 Factor NZ  : 2.100e+02
 Factor Ops : 2.870e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.67645245e+06 -4.99435609e+06  3.47e+03 0.00e+00  9.84e+05     0s
   1   1.40930140e+06 -1.58740897e+06  3.47e-03 0.00e+00  1.20e+05     0s
   2   1.86695636e+05 -2.51501497e+05  3.47e-09 0.00e+00  1.75e+04     0s
   3   2.08575874e+04 -4.24957571e+04  3.91e-14 2.84e-14  2.53e+03     0s
   4 

:Optimal

>**\[Solution\]**: Sparse Linear Regression

> One way of expressing it in JuMP is through the following formulation (using binary variables and big-$M$ constraints):

> $$
\underset{x}{\min}\ \sum_i z_i^2 \\
\text{s.t.}\quad z_i = a_i^\top x - b \quad\forall i \\
x_j \leq My_j \\
x_j \geq -My_j \\
\sum_{j=1}^p y_j \leq k \\
y_j\in\{0,1\}\quad\forall j=1,\dots,p
$$

> where $a_i^\top$ is the $i$-th row of the data matrix $A$.

In [12]:
using JuMP, Gurobi

M = 10000
k = 10

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, y[1:p], Bin)
@variable(m, z[1:n])

@objective(m, Min, sum(z[i]^2 for i in 1:n))

@constraint(m, [i=1:n], z[i] == sum(A[i,j]*x[j] for j in 1:p) - b[i])

@constraint(m,[j=1:p], x[j] <= M*y[j])
@constraint(m,[j=1:p], x[j] >= -M*y[j])
@constraint(m, sum(y[j] for j in 1:p) <= k)
        
solve(m)

Optimize a model with 31 rows, 30 columns and 145 nonzeros
Model has 20 quadratic objective terms
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [9e-03, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 2e+01]
Presolve removed 11 rows and 5 columns
Presolve time: 0.00s
Presolved: 20 rows, 25 columns, 120 nonzeros
Presolved model has 20 quadratic objective terms
Variable types: 25 continuous, 0 integer (0 binary)
Presolve time: 0.00s
Presolved: 20 rows, 25 columns, 120 nonzeros
Presolved model has 20 quadratic objective terms
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.900e+02
 Factor NZ  : 2.100e+02
 Factor Ops : 2.870e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.32331769e+09 

:Optimal

In [13]:
getvalue(x)

5-element Array{Float64,1}:
 7.85979
 9.8411 
 6.95982
 3.84803
 3.20848

>**\[Solution\]**: Information Callbacks

In [9]:
using JuMP, Gurobi

M = 10000
k = 2

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, y[1:p], Bin)
@variable(m, z[1:n])

@objective(m, Min, sum(z[i]^2 for i in 1:n))

@constraint(m, z .== A*x - b)
@constraint(m,[j=1:p], x[j] <= M*y[j])
@constraint(m,[j=1:p], x[j] >= -M*y[j])
@constraint(m, sum(y[j] for j in 1:p) <= k)

### --- NEW ---
node = []
obj = []
bestbound = []
function infocallback(cb)
    push!(node, MathProgBase.cbgetexplorednodes(cb))
    push!(obj, MathProgBase.cbgetobj(cb))
    push!(bestbound, MathProgBase.cbgetbestbound(cb))
end
addinfocallback(m, infocallback, when = :Intermediate)
### END OF NEW

solve(m)

Optimize a model with 31 rows, 30 columns and 145 nonzeros
Model has 20 quadratic objective terms
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [9e-03, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 26 rows, 30 columns, 135 nonzeros
Presolved model has 20 quadratic objective terms
Variable types: 25 continuous, 5 integer (5 binary)

Root relaxation: objective 2.196497e+00, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.19650    0    5          -    2.19650      -     -    0s
H    0     0                     415.3988979    2.19650  99.5%     -    0s
H    0     0                     356.2185398    2.19650  99.4%     -    0s
    

:Optimal

In [10]:
hcat(node, obj, bestbound)

19×3 Array{Float64,2}:
  0.0    1.0e100   -9.12658e6
  0.0    1.0e100   -9.12658e6
  0.0    1.0e100    2.1965   
  0.0  356.219      2.1965   
  0.0  356.219      2.1965   
  0.0  356.219      2.1965   
  0.0  356.219      2.1965   
  0.0  356.219      2.1965   
  1.0  356.219      2.1965   
  2.0  356.219      2.1965   
  3.0  356.219      2.1965   
  4.0  356.219      2.1965   
  5.0  356.219     81.2699   
  6.0  356.219     81.2699   
  7.0  356.219     81.2699   
  8.0  356.219     81.2699   
  9.0  189.036    149.356    
 10.0  189.036    149.356    
 11.0  189.036    149.356    

>**\[Solution\]**: Lazy Least Squares

In [14]:
using JuMP, Gurobi

M = 10000
k = 2

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, y[1:p], Bin)
@variable(m, z[1:n])
@variable(m, eta >= 0)

@objective(m, Min, eta)

@constraint(m, [i=1:n], z[i] == sum(A[i,j]*x[j] for j in 1:p) - b[i])
@constraint(m,[j=1:p], x[j] <= M*y[j])
@constraint(m,[j=1:p], x[j] >= -M*y[j])
@constraint(m, sum(y[j] for j in 1:p) <= k)

### --- NEW ---
function lazyleastsqs(cb)
    curr_z = getvalue(z)
    @lazyconstraint(cb, eta >= norm(curr_z)^2 + 2*sum(curr_z[i]*(z[i] - curr_z[i]) for i in 1:n))
end
addlazycallback(m, lazyleastsqs)
### END OF NEW

solve(m)

Optimize a model with 31 rows, 31 columns and 145 nonzeros
Variable types: 26 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [9e-03, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 25 rows and 20 columns
Presolve time: 0.00s
Presolved: 6 rows, 11 columns, 15 nonzeros
Variable types: 6 continuous, 5 integer (5 binary)

Root relaxation: objective 0.000000e+00, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    2          -    0.00000      -     -    0s
     0     0    0.00000    0    2          -    0.00000      -     -    0s
     0     0    0.00000    0    2          -    0.00000      -     -    0s
     0     0    0.00000    0    2          -    0.00000      -     -    0s
     0     0    0.00000    0    -          -    0.000

:Optimal